# Clustering Assignment

## Part-1

## Requirements for the Project:

For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.
Start by creating a new Notebook for this assignment. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe:

## Pre-processing Stage :

## Data Collection/Web Scrapping

In [ ]:
#install Beautiful Soup and requests for Web Scaping
!pip install BeautifulSoup4
!pip install requests

In [ ]:
# importing necessary libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [ ]:
#get html from wiki page and create soup object
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(source.text, 'lxml')

#using soup object, iterate the .wikitable to get the data from the HTML page and store it into a list
data = []
columns = []
table = soup.find(class_='wikitable')
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    
    #First row of data is the header
    if (index == 0):
        columns = section
    else:
        data.append(section)

#convert list into Pandas DataFrame
canada_df = pd.DataFrame(data = data,columns = ["PostalCode", "Borough", "Neighborhood"])
canada_df.head()

## Data Cleaning

1. Remove Boroughs that are 'Not assigned'
2. More than one neighborhood can exist in one postal code area, combined these into one row with the neighborhoods separated with a comma
3. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [ ]:
#Remove Boroughs that are 'Not assigned'
canada_df = canada_df[canada_df['Borough'] != 'Not assigned']
canada_df.head()

In [ ]:

# More than one neighborhood can exist in one postal code area, combined these into one row with the neighborhoods separated with a comma
canada_df["Neighborhood"] = canada_df.groupby("PostalCode")["Neighborhood"].transform(lambda neigh: ', '.join(neigh))

#remove duplicates
canada_df = canada_df.drop_duplicates()

#update index to be postalcode if it isn't already
if(canada_df.index.name != 'PostalCode'):
    canada_df = canada_df.set_index('PostalCode')
    
canada_df.head()

In [ ]:
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
canada_df['Neighborhood'].replace("Not assigned", canada_df["Borough"],inplace=True)
canada_df.head()

## Output

In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [ ]:
canada_df.shape

## Part-2

## This is the Part-2 of the Capstone Project

## Add Geospatial Data

In [ ]:
#using Geocoder Package
!pip install geopy

In [ ]:
url = "https://raw.githubusercontent.com/ansafjunaid/Coursera_Capstone/master/Geospatial_Coordinates.csv"
df_geo_coor = pd.read_csv(url)
df_geo_coor.head()

We need to couple 2 dataframes "canada_df" and "df_geo_coor" into one dataframe.

In [ ]:
df_toronto = pd.merge(canada_df, df_geo_coor, how='left', left_on = 'PostalCode', right_on = 'Postal Code')
# remove the "Postal Code" column
df_toronto.drop("Postal Code", axis=1, inplace=True)
df_toronto.head()